In [1]:
set SV_BATCH_MODE 1; set env(SV_BATCH_MODE) 1; set SV_RELEASE_BUILD 1

1


In [2]:
source $env(SV_HOME)/Tcl/SimVascular_2.0/simvascular_startup.tcl

Loading vtk procs...
assuming vtk 8.1
  TclVtk:      Dynamic Libs (not_loaded: 0)
Done loading vtk procs...
ERROR: could not find registry key:
HKEY_LOCAL_MACHINE\SOFTWARE\SimVascular\SimVascular\2019-02-26 RunDir
Found svPre (C:\Program Files\SimVascular\svSolver\2019-01-25\svpre-bin.exe)
Found svPost (C:\Program Files\SimVascular\svSolver\2019-01-25\svpost-bin.exe)
Found svSolver (C:\Program Files\SimVascular\svSolver\2019-01-25\svsolver-msmpi-bin.exe)


In [3]:
solid_setKernel -name PolyData
mesh_setKernel -name TetGen
set gOptions(meshing_kernel) TetGen
set gOptions(meshing_solid_kernel) PolyData

PolyData
TetGen
TetGen
PolyData


In [4]:
set num_procs 4
set selected_LS svLS
set timesteps 64

4
svLS
64


In [5]:
  if {[info vars PRESOLVER] == ""} {
      set PRESOLVER     $gExternalPrograms(svpre)
  }
  if {[info vars POSTSOLVER] == ""} {
      set POSTSOLVER    $gExternalPrograms(svpost)
  }
  if {[info vars MPIEXEC] == ""} {
	set MPIEXEC       $gExternalPrograms(mpiexec)
  }
  if {[info vars SOLVER] == ""} {
	set SOLVER        $gExternalPrograms(svsolver-mpi)
  }
  if {[info vars FLOWSOLVER_CONFIG] == ""} {
	set FLOWSOLVER_CONFIG [file dirname $gExternalPrograms(svsolver-mpi)]
  }


C:/Program Files/SimVascular/svSolver/2019-01-25/svpre-bin.exe
C:/Program Files/SimVascular/svSolver/2019-01-25/svpost-bin.exe
C:/Program Files/Microsoft MPI//bin/mpiexec.exe
C:/Program Files/SimVascular/svSolver/2019-01-25/svsolver-msmpi-bin.exe
C:/Program Files/SimVascular/svSolver/2019-01-25


In [6]:
proc cylinder_create_model_PolyData {dstdir} {

  # just copy the model for now

  catch {repos_delete -obj cyl}
  solid_readNative -file cylinder.vtp -obj cyl

  file copy cylinder.vtp $dstdir
  file copy cylinder.vtp.facenames $dstdir
  
  return [file join $dstdir cylinder.vtp]

}


In [9]:
proc steady_cylinder_create_mesh_TetGen {solidfn dstdir} {

  #
  #  Mesh the solid
  #

  puts "Creating mesh."

  # create meshsim style script file
  set fp [open [file join $dstdir cylinder.tgs] w]
  fconfigure $fp -translation lf
  puts $fp "msinit"
  puts $fp "logon [file join $dstdir cylinder.logfile]"
  puts $fp "loadModel $solidfn"
  puts $fp "setSolidModel"
  puts $fp "newMesh"
  puts $fp "option surface 1"
  puts $fp "option volume 1"
  puts $fp "option GlobalEdgeSize 0.5"
  puts $fp "wallFaces wall"
  puts $fp "option QualityRatio 1.4"
  puts $fp "option NoBisect 1"
  puts $fp "generateMesh"
  puts $fp "writeMesh [file join $dstdir cylinder.sms] vtu 0"
  puts $fp "deleteMesh"
  puts $fp "deleteModel"
  puts $fp "logoff"
  close $fp

  catch {repos_delete -obj mymesh}
  mesh_readTGS [file join $dstdir cylinder.tgs] mymesh

  puts "Writing out mesh surfaces."
  file mkdir [file join $dstdir mesh-complete]
  file mkdir [file join $dstdir mesh-complete mesh-surfaces]

   mesh_writeCompleteMesh mymesh cyl cylinder [file join $dstdir mesh-complete]

}

In [10]:
proc steady_cylinder_create_flow_files_generic {dstdir} {

  # Write steady flowrate
  file mkdir [file join $dstdir flow-files]
  set fp [open [file join $dstdir flow-files inflow.flow] "w"]
  puts $fp "\#  Time (sec)   Flow (mm^3/sec)"
  puts $fp "0   -1570.796327"
  puts $fp "0.2 -1570.796327"
  close $fp

}

proc steady_cylinder_create_bc_files_generic {solidfn dstdir} {

  #
  #  Create an analytic Inflow Waveform and create a flow file.
  #  Also calculate the FFT of the waveform for later.
  #

  steady_cylinder_create_flow_files_generic $dstdir
  
  puts "Generating sinusodial volumetric flow waveform."
  set viscosity 0.004
  set density 0.00106
  set T 0.2
  set Vbar 135
  set radius 2
  set omega [expr 2.0*[math_pi]/$T]

  puts "Write bct.dat and bct.vtp files."
  global gBC
  global guiABC
  global gFilenames

  # load in the solid used for meshing
  # set both vars to handle any type of model
  set gFilenames(atdb_solid_file) $solidfn
  set gFilenames(polydata_solid_file) $solidfn
  wormGUIloadSolidModel

  # set params
  set gBC(period) $T
  set guiABC(viscosity) $viscosity
  set guiABC(density) $density
  set guiABC(type_of_profile) plug

  set guiABC(mesh_face_file) ""
  set guiABC(flow_rate_file) ""
  set guiABC(bct_dat_file)   ""
  set guiABC(bct_vtp_file)   ""
  set guiABC(face_name) {$facename}
  set guiABC(mesh_face_file) [file join $dstdir mesh-complete mesh-surfaces \$facename.vtp]
  set guiABC(flow_rate_file) [file join $dstdir flow-files \$facename.flow]
  set guiABC(bct_dat_file)   [file join $dstdir bct.dat.\$facename]
  set guiABC(bct_vtp_file)   [file join $dstdir bct.vtp.\$facename]

  # write files
  wormGUIwriteMultipleFaces
}


In [14]:
set guiMMvars(meshGenerateVolumeMesh) 1

cyl /tmp/solid/pd mymesh


In [10]:
cd C:/tmp;pwd

C:/tmp


In [16]:
set rundir [clock format [clock seconds] -format "%m-%d-%Y-%H%M%S"]
set fullrundir [file join [pwd] $rundir]
file mkdir $fullrundir

if {$num_procs == 1} {
  set fullsimdir $fullrundir
} else {
  set fullsimdir [file join $fullrundir $num_procs-procs_case]
}

set solidfn [cylinder_create_model_$gOptions(meshing_solid_kernel) $fullrundir]
steady_cylinder_create_mesh_$gOptions(meshing_kernel) $solidfn $fullrundir

03-04-2019-172311
C:/tmp/03-04-2019-172311
C:/tmp/03-04-2019-172311/4-procs_case
C:/tmp/03-04-2019-172311/cylinder.vtp
Creating mesh.
TetGen
TetGen
line: msinit
ignoring line: <msinit>
line: logon C:/tmp/03-04-2019-172311/cylinder.logfile
line: loadModel C:/tmp/03-04-2019-172311/cylinder.vtp
Filename: C:/tmp/03-04-2019-172311/cylinder.vtp
sourcing C:/tmp/03-04-2019-172311/cylinder.vtp.facenames


In [17]:
repos_list


line: setSolidModel
line: newMesh
Converting to TetGen...
Converting Points...
Converting Faces...
Adding Facet Markers...
line: option surface 1
line: option volume 1
line: option GlobalEdgeSize 0.5
line: wallFaces wall
name_id: 2
line: option QualityRatio 1.4
line: option NoBisect 1
line: generateMesh
Remeshing surface with MMG...
TetGen Meshing Started...
Delaunizing vertices...
Delaunay seconds:  0.037
Creating surface mesh ...
Surface mesh seconds:  0.009
Recovering boundaries...
Boundary recovery seconds:  0.014
Removing exterior tetrahedra ...
Exterior tets removal seconds:  0.005
Recovering Delaunayness...
Delaunay recovery seconds:  0.006
Refining mesh...
Refinement seconds:  0.104
Optimizing mesh...
Optimization seconds:  0.011

Writing nodes.
Writing elements.
Writing faces.
Writing edges.
Writing neighbors.

Output seconds:  0.003
Total running seconds:  0.19

Statistics:

  Input points: 3235
  Input facets: 6466
  Input holes: 0
  Input regions: 0

  Mesh points: 9159
  M

In [18]:
repos_list

Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10
Final mesh improvement
Converting to TetGen...
Converting Points...
Converting Faces...
Adding Facet Markers...
Converting Points to VTK Structures...
Converting Elements to VTK Structures...
line: writeMesh C:/tmp/03-04-2019-172315/cylinder.sms vtu 0
line: deleteMesh
ignoring line: <deleteMesh>
line: deleteModel
ignoring line: <deleteModel>
line: logoff
Writing out mesh surfaces.
faceinfo: 1 1 {}
faceinfo: 2 2 {}
faceinfo: 3 3 {}
1
cyl /tmp/solid/pd mymesh


In [19]:
repos_list


cyl /tmp/solid/pd mymesh


In [22]:
info commands repos_*

repos_clearLabel repos_getstring repos_writeVtkStructuredPoints repos_load repos_delete repos_save repos_delete_wrapper repos_importVtkImg repos_keyInUse repos_getLabel repos_getObjTok repos_importVtkPd repos_importVtkSp repos_sorted repos_exists repos_readVtkPolyData repos_subList repos_readXMLPolyData repos_getLabelKeys repos_writeVtkPolyData repos_writeXMLPolyData repos_exportToVtk repos_writeVtkUnstructuredGrid repos_type repos_setLabel repos_setstring repos_list repos_importVtkUnstructuredGrid repos_deleteList


In [24]:
glob *

.ipynb_checkpoints 02-08-2019-210136 02-08-2019-210425 02-08-2019-210507 03-04-2019-171418 03-04-2019-172311 03-04-2019-172315 2019-03-04-juypter_examples cylinder.vtp cylinder.vtp.facenames manually-generated-tests nate.vtp python-cube-render.ipynb simvascular_small.tcl tcl-devel.ipynb tcl-vis-cube-for-fanwei.ipynb tcl-vis-cube.ipynb tcl.tcl Untitled.ipynb Untitled1.ipynb Untitled10.ipynb Untitled100.ipynb Untitled101.ipynb Untitled102.ipynb Untitled103.ipynb Untitled104.ipynb Untitled105.ipynb Untitled11.ipynb Untitled12.ipynb Untitled13.ipynb Untitled14.ipynb Untitled15.ipynb Untitled16.ipynb Untitled17.ipynb Untitled18.ipynb Untitled19.ipynb Untitled2.ipynb Untitled20.ipynb Untitled21.ipynb Untitled22.ipynb Untitled23.ipynb Untitled24.ipynb Untitled25.ipynb Untitled26.ipynb Untitled27.ipynb Untitled28.ipynb Untitled29.ipynb Untitled3.ipynb Untitled30.ipynb Untitled31.ipynb Untitled32.ipynb Untitled33.ipynb Untitled34.ipynb Untitled35.ipynb Untitled36.ipynb Untitled37.ipynb Untitled

In [15]:
vis_initgr "Nathan"

invalid command name "cmdExists"


In [16]:
cmdExists foo

invalid command name "cmdExists"


In [17]:
vis_init

In [18]:
info proc

seg_3DRenForScreen vis_splineImageWidgetRm RCR_parameter_estimator math_scaleVec dcm_img2_readSliceROI guiTGmeshCoarsenPercent vtp_calc_avg_pressure vis_planeWidgetOn vis_interactorSpecialRotate tmpobj PickSeedNoCrosshairs pkg_mkIndex vis_volShadeOff vis_splineClosedWidgetGetSamplePts vis_nodeRmVtk vis_gridSetDataOrigin guiMMextractBoundaries vtu_extractSingleFace auto_reset pause vis_renRestoreCamFromFile vis_imgRepos vis_movieSaveCameraStatesToFile vis_planeWidgetRm RCR_utility_optimize_func vis_splineClosedWidgetUpdatePtsPlaneOnly solid_mkDiscreteModel tk_textPaste vis_nodeAddRepos math_magnitude vis_interactorB1release vis_splineClosedWidgetRmSlice vis_boxWidgetGetEnabled vis_vGetActor vis_sphereWidget2GetEnabled vis_pExists vis_boxWidgetGetPlanes geom_densify math_matrix_mult_3x1 file_dos2unix svView guiTGmeshOptimization lset2_getMinH vis_sphereWidgetScaleOff vis_pGetDisplayedActors2D seg_writeSliceSegEdgePNG dicom_createPatientFile PolyDataVMTKCenterlines vis_interactorToggle vi

In [19]:
info foo

unknown or ambiguous subcommand "foo": must be args, body, class, cmdcount, commands, complete, coroutine, default, errorstack, exists, frame, functions, globals, hostname, level, library, loaded, locals, nameofexecutable, object, patchlevel, procs, script, sharedlibextension, tclversion, or vars


In [21]:
info commands solid_readNative2

In [22]:
proc cmdExists {cmd} {
  if {[info commands $cmd] != ""} {
    return 1
  } else {
    return 0
  }
}

In [23]:
cmdExists foo

0


In [24]:
cmdExists repos_list

1


In [25]:
vis_init

g8bitGrayscaleLUT: a vtk object with that name already exists.


In [27]:
vis_initgr seth

seth_ren1


In [29]:
generalView seth_ren1 nate


/tmp/svView/nate
